In [1]:
from categories_download import get_categories
from requests import get
from bs4 import BeautifulSoup,NavigableString
import pandas as pd
import os
import re
import urllib.request, urllib.error, urllib.parse
import sys

In [ ]:
class recipe_info:
    def __init__(self):
        self.recipe_name = None
        self.recipe_url = None
        self.recipe_img_link = None
        self.recipe_rating = None
        self.recipe_difficulty = None
        self.recipe_time = None
        self.preparation_time = None
        self.recipe_author = None
        self.recipe_steps = None
        self.recipe_ingredients = None
        self.recipe_portion = None
        self.recipe_accessories = None
        self.number_vote = None
        self.number_favorites = None
        self.variation_link = None
        self.variation_name = None
        self.machine_name = None
        self.machine_addtional_info = None
        self.addtional_categories_dic = None
        self.addtional_categories_name = None
        self.tips = None
        self.iftested = None
        
    def item_info_download(self,item):
        #download author
        author_container = item.find_all('div',class_='item-author')[0]
        self.recipe_author = author_container.a.text
#         print(item_author)
        #download rating
        item_rating_container = item.find('div',class_ = 'item-rating clearfix')
        if (item_rating_container != None):
            self.recipe_rating = item_rating_container.div['data-average']
#             print(self.recipe_rating)
            #downloadd number of vote
            if (item_rating_container.span != None):
                vote_string = item_rating_container.span.text.strip()
                self.number_vote = re.findall('[0-9]+', vote_string)

        favourite_container = item.find('div',class_ = 'col-xs-2 favourite-inf')
        self.number_favorites = favourite_container.text.strip()
#         print(self.favorites)
        #download difficulty
        item_difficulty_container = item.find('div',class_ = 'col-xs-3 difficulty-inf')
        self.recipe_difficulty = item_difficulty_container.text.strip()
#         print(item_difficulty)
        #download cooking time
        item_time_container = item.find('div',class_ = 'col-xs-3 cooking-time-inf')
        self.recipe_time = item_time_container.text.strip()
#         print(item_time)
        #download name
        self.recipe_name = item.a['title'].replace('/','')
#         print(item_name)
        #download image link
        self.recipe_img_link = item.img['src']
#         print(item_img_link)
        #download item url
        item_url_add = item.a['href']
        self.recipe_url = home_url+item_url_add
#         print(item_extend_url)
        

    def insert_icon_text(self,step):
        try:
            img_string = step.img
            operation = '#'+img_string['title']+'#'
            step_icon = str(step).replace(str(img_string),operation)
            step_icon_soup = BeautifulSoup(step_icon, 'html.parser')
            return step_icon_soup.text.strip().replace('\xa0',' ')
        except:
            step_image_url = step.img['src']
            return " ".join(step.text.split()) + "$" + step_image_url+"$"

    
    def download_recipe_steps_ingredients(self, step_list,ingredient_list):
        self.recipe_steps = []
        if len(step_list) == 1:
            try:
#                 print(insert_icon_text(step_list[0]))
                self.recipe_steps.append(insert_icon_text(step_list[0]))
            except:
                step_list_container = step_list[0].find_all('p')
                if (len(step_list_container) > 1):
                    for step in step_list_container:
                        step_text = step.text.strip()
                        if step_text != '':
                            self.recipe_steps.append(step_text.replace('\xa0',' '))
                elif(sum(1 for e in step_list[0].childGenerator())>4):
                    for step in step_list[0].childGenerator():
                        if not (step and isinstance(step,NavigableString)):
                            continue
                        elif(str(step).strip() ==''):
                            continue
                        else:
                            self.recipe_steps.append(str(step).strip())
                else:
                    self.recipe_steps.append(" ".join(step_list[0].text.split()))
        else:
            for i in range(len(step_list)):
                if (step_list[i].img != None):
                    self.recipe_steps.append(self.insert_icon_text(step_list[i]))
                else:                
                    self.recipe_steps.append(" ".join(step_list[i].text.split()))   
                
        #append ingredients to dictionary
        self.recipe_ingredients = []
        for i in range(len(ingredient_list)):
             self.recipe_ingredients.append(" ".join(ingredient_list[i].text.split()))
        
    def recipe_download_details(self):
        recipe_response = get(self.recipe_url)
        recipe_html_soup = BeautifulSoup(recipe_response.text, 'html.parser')
        
        #download variations 
        variations_container = recipe_html_soup.find('div',class_= 'col-sm-12 sidebar-box variant-box')
        if (variations_container != None):
            self.variation_link = home_url+variations_container.a['href']
            self.variation_name = variations_container.a.text
        
        #find recipe steps list
        step_container = recipe_html_soup.find('ol', class_ = 'steps-list')
        step_list = step_container.find_all('li')
        
        #find tips 
        tips_container = recipe_html_soup.find('div',class_='tips')
#         print(tips_container.find('p'))
        tips = tips_container.find('p')
        if(tips!=None):
            self.tips = tips.text
        #find ingredients list and portion
        ingredients_container = recipe_html_soup.find('div', class_ = 'ingredients')
        ingredient_list = ingredients_container.find_all('li')
        if (ingredients_container.find('p',class_='padding-bottom-10')):
            self.recipe_portion = ingredients_container.find('p',class_='padding-bottom-10').text.strip()

        #download if tested 
        iftested_container = recipe_html_soup.find('span','recipe-testing-status-text')
        self.iftested = re.findall('\w+\stested', iftested_container.text)[0]
        
        #process recipe step and ingredient list
        self.download_recipe_steps_ingredients(step_list,ingredient_list)
        
        #accessories list download
        accessories_container = recipe_html_soup.find('div', class_ = 'accessories-list')
        if accessories_container == None:
            self.accessories = []
        else:
            tools_list = accessories_container.find_all("meta")
            accessories = []
            for tool in tools_list:
                accessories.append(tool.get("content"))
            self.recipe_accessories = [accessories]
        
        #download addtional info
        self.download_addtional_info(recipe_html_soup)
       
    def download_addtional_info(self,recipe_html_soup):
        addtional_info_container = recipe_html_soup.find('div',class_= 'additional-info')
        addtion_info_list = addtional_info_container.find_all('li')
        #preparation time dowload
        time_container = addtion_info_list[0]
        preparation_time_container = time_container.find_all('div',class_='smallText')
        
        if (preparation_time_container):
            self.preparation_time = preparation_time_container[0].find("span", id="preparation-time-final").text.strip()
            #cooking method
            cooking_method = preparation_time_container[1].text.strip()
        #machine and machine addtional info
        machine_contianer = addtion_info_list[2]
        self.machine_name = machine_contianer.h5.text
        self.machine_addtional_info = machine_contianer.find('div',class_="margin-top-10")
        if(self.machine_addtional_info!=None):
            self.machine_addtional_info = self.machine_addtional_info.text.strip()
        #addtional categroies
        addtional_categories_container = addtion_info_list[4]
        addtional_categories_list =addtional_categories_container.find_all('a',class_='catText preventDefault')
        self.addtional_categories_dic ={}
        self.addtional_categories_name = []
        for category in addtional_categories_list:
            self.addtional_categories_name.append(category.text)
            self.addtional_categories_dic[category.text] = category['href']
        
    def create_data_frame(self):
        #create data frame
        item_dic = {}
        item_dic['link'] = self.recipe_url
        item_dic['img link']= self.recipe_img_link
        item_dic['rating'] = self.recipe_rating
        item_dic['number vote'] = self.number_vote
        item_dic['difficulty'] = self.recipe_difficulty
        item_dic['cooking time'] = self.recipe_time
        item_dic['preparation time'] = self.preparation_time
        item_dic['author'] = self.recipe_author
        item_dic['steps'] = [self.recipe_steps]
        item_dic['tips'] = self.tips
        item_dic['ingredients'] = [self.recipe_ingredients]
        item_dic['accessories'] = self.recipe_accessories
        item_dic['number favorite'] = self.number_favorites
        item_dic['variation link'] = self.variation_link
        item_dic['variation name'] = self.variation_name
        item_dic['portion'] = self.recipe_portion
        item_dic['recipe created for'] = self.machine_name
        item_dic['machine addtional info'] = self.machine_addtional_info
#         item_dic['addtional categories'] = self.addtional_categories_dic
        item_dic['addtional categories'] = [self.addtional_categories_name]
        item_dic['if tested'] = self.iftested
        return item_dic
    
    
    def download_html(self,path):
        #save html to file
        response = urllib.request.urlopen(self.recipe_url)
        webContent = response.read()

        f = open(path+'/'+self.recipe_name+'.html', 'wb')
        f.write(webContent)
        f.close

In [3]:
home_url = "https://www.recipecommunity.com.au"
categories_info_file = get_categories()
categories_data = pd.read_csv(categories_info_file, index_col=0)

/******************************/
finish categories info download
/******************************/


In [ ]:
dataset_columns = ['link','img link','rating','number vote','number favorite',
                   'difficulty','cooking time','portion','preparation time',
                   'author','accessories','steps','tips','ingredients','if tested','recipe created for',
                   'machine addtional info','addtional categories','variation link','variation name']


categories_indices = categories_data.index
counter = 0
for category_index in categories_indices:
    print('/*******************************/')
    print('Start Download Category '+ str(category_index))
    print('/*******************************/')
    #creat folders and init file name
    os.mkdir(str(category_index))
    category_recipe_outfile = str(category_index) + '/'+str(category_index)+'.csv' 
    items_data = pd.DataFrame(columns=dataset_columns)
    #get item url
    category_url = categories_data['link'][category_index]
#     print(item_url)
    category_response = get(category_url)
    category_html_soup = BeautifulSoup(category_response.text, 'html.parser')
    
    pager_container = category_html_soup.find('div',class_= 'pager')
    lastpage_extend = pager_container.a['href']
    pager_number = re.findall('[0-9]+', lastpage_extend)[0]
    pager_number = int(pager_number)
    
    try:
    
        for current_page in range(1,pager_number+1): 

            page_url= category_url + '?page=' + str(current_page)
            print(page_url)

            #download url contents and apply beautifulsoup
            item_response = get(page_url)
            item_html_soup = BeautifulSoup(item_response.text, 'html.parser')


            #find items on the first page
            items_container = item_html_soup.find_all('div', class_ = 'thumbnail result-recipe result-grid-display')
        #     print(len(items_container))

            for item in items_container:
                #init recipe class
                recipe_class = recipe_info()
                recipe_class.item_info_download(item)
                print("downloading "+ recipe_class.recipe_name)

                #download html
                html_path = str(category_index)
                recipe_class.download_html(html_path)

        #         #process recipe url
                recipe_class.recipe_download_details()

                #create recipe data frame
                item_dic = recipe_class.create_data_frame()
                item_info = pd.DataFrame(item_dic,index=[recipe_class.recipe_name])
                items_data = items_data.append(item_info)
    #             counter = counter +1
    #             if counter >= 1:
    #                 break
        print('/*******************************/')
        print('Category '+ str(category_index)+' saved')
        print('/*******************************/')
        items_data.to_csv(category_recipe_outfile, index = True,sep=",")
            #finish current pager crawler
        #finish all pages
#         break
    except:
        items_data.to_csv(category_recipe_outfile, index = True,sep=",")
        print("error ocurred")

/*******************************/
Start Download Category Baby food
/*******************************/
https://www.recipecommunity.com.au/categories/baby-food?page=1
downloading Baby Biscuits (sugar free)
downloading Baby coconut chicken meal
1. Place chicken, vegetables and coconut milk into TM bowl.2. Cook for 10MIN - 100DEG -#Counter-clockwise operation#3. Pulverise for about 10SECS - SP7, checking there are no lumps. (or try for just 5SECS for a chunkier meal).The fats in coconut milk have been found to be close those in breast milk and thus is an ideal way to keep meat moist for your baby as well as being nutritious. This meal can be frozen (after using fresh chicken) and made thinner by adding more coconut milk, milk, breast milk if required.
downloading Apple, Pear and Cinnamon Puree (Baby Food 6+ months)
downloading Raw cacao bites
downloading 'Sausage' rolls
downloading Baby Rice Rusks
downloading Apricot Chicken Puree (Baby Food 8+ months)
downloading Fruity vegetables
error o

In [20]:
dataset_columns = ['link','img link','rating','number vote','number favorite',
                   'difficulty','cooking time','portion','preparation time',
                   'author','accessories','steps','tips','ingredients','if tested','recipe created for',
                   'machine addtional info','addtional categories','variation link','variation name']
items_data = pd.DataFrame(columns=dataset_columns)

page_url= "https://www.recipecommunity.com.au/categories/baby-food?page=1"
print(page_url)

#download url contents and apply beautifulsoup
item_response = get(page_url)
item_html_soup = BeautifulSoup(item_response.text, 'html.parser')


#find items on the first page
items_container = item_html_soup.find_all('div', class_ = 'thumbnail result-recipe result-grid-display')
#     print(len(items_container))

for item in items_container:
    #init recipe class
    recipe_class = recipe_info()
    recipe_class.item_info_download(item)
    print("downloading "+ recipe_class.recipe_name)

#         #process recipe url
    recipe_class.recipe_download_details()

    #create recipe data frame
    item_dic = recipe_class.create_data_frame()
    item_info = pd.DataFrame(item_dic,index=[recipe_class.recipe_name])
    items_data = items_data.append(item_info)

https://www.recipecommunity.com.au/categories/baby-food?page=1
downloading Baby Biscuits (sugar free)
downloading Baby coconut chicken meal
1. Place chicken, vegetables and coconut milk into TM bowl.2. Cook for 10MIN - 100DEG -#Counter-clockwise operation#3. Pulverise for about 10SECS - SP7, checking there are no lumps. (or try for just 5SECS for a chunkier meal).The fats in coconut milk have been found to be close those in breast milk and thus is an ideal way to keep meat moist for your baby as well as being nutritious. This meal can be frozen (after using fresh chicken) and made thinner by adding more coconut milk, milk, breast milk if required.
downloading Apple, Pear and Cinnamon Puree (Baby Food 6+ months)
downloading Raw cacao bites
downloading 'Sausage' rolls
downloading Baby Rice Rusks
downloading Apricot Chicken Puree (Baby Food 8+ months)
downloading Fruity vegetables
downloading Red Lentil and Sweet Potato Puree (From 6 months)
Place onion, celery, zucchini and carrot in TM 

In [17]:
def insert_icon_text(step):
    try:
        img_string = step.img
        operation = '#'+img_string['title']+'#'
        step_icon = str(step).replace(str(img_string),operation)
        step_icon_soup = BeautifulSoup(step_icon, 'html.parser')
        return step_icon_soup.text.strip().replace('\xa0','')

    except:
        step_image_url = step.img['src']
        return " ".join(step.text.split()) + ".$" + step_image_url+"$"
page_url= "https://www.recipecommunity.com.au/baby-food-recipes/baby-coconut-chicken-meal/4wyfnaab-f9c28-481590-cfcd2-xs7vp6mu"
# page_url= "https://www.recipecommunity.com.au/baby-food-recipes/brown-rice-and-quinoa-baby-rusks/ff5c7zsn-b0b2e-764829-cfcd2-dqad3va8"
print(page_url)

#download url contents and apply beautifulsoup
item_response = get(page_url)
recipe_html_soup = BeautifulSoup(item_response.text, 'html.parser')
#find recipe steps list
step_container = recipe_html_soup.find('ol', class_ = 'steps-list')
step_list = step_container.find_all('li')
    
recipe_steps = []
if len(step_list) == 1:
    try:
        recipe_steps.append(insert_icon_text(step_list[0]))
    except:
        step_list_container = step_list[0].find_all('p')
        if (len(step_list_container) > 1):
            for step in step_list_container:
                step_text = step.text.strip()
                if step_text != '':
                    recipe_steps.append(step_text)
        elif(sum(1 for e in step_list[0].childGenerator())>4):
            for step in step_list[0].childGenerator():
                if not (step and isinstance(step,NavigableString)):
                    continue
                elif(str(step).strip() ==''):
                    continue
                else:
                    recipe_steps.append(str(step).strip())
        else:
            for i in range(len(step_list)):
                recipe_steps.append(" ".join(step_list[i].text.split()))

else:
    for i in range(len(step_list)):
        if (step_list[i].img != None):
            recipe_steps.append(insert_icon_text(step_list[i]))
        else:
            recipe_steps.append(" ".join(step_list[i].text.split()))
print(recipe_steps)

https://www.recipecommunity.com.au/baby-food-recipes/baby-coconut-chicken-meal/4wyfnaab-f9c28-481590-cfcd2-xs7vp6mu
['1. Place chicken, vegetables and coconut milk into TM bowl.2. Cook for 10MIN - 100DEG -#Counter-clockwise operation#3. Pulverise for about 10SECS - SP7, checking there are no lumps. (or try for just 5SECS for a chunkier meal).The fats in coconut milk have been found to be close those in breast milk and thus is an ideal way to keep meat moist for your baby as well as being nutritious. This meal can be frozen (after using fresh chicken) and made thinner by adding more coconut milk, milk, breast milk if required.']


In [16]:
step_list[4]

<li>
<div class="media">
<div class="pull-left">
<img alt="" class="recipe-steps-image" data-holder-rendered="true" data-src="holder.js/150x150" src="https://d2mkh7ukbp9xav.cloudfront.net/recipeimage/qlhgka4k-74388-924687-cfcd2-vtu0swek/19f392c7-eb66-4fa8-9825-34cc78cc272f/medium/baby-sausage-scrolls.jpg" width="150">
<div class="stop-propagation">
<span class="pinterest">
<a data-pin-count="beside" data-pin-do="buttonPin" href="https://www.pinterest.com/pin/create/button/?url=https%3A%2F%2Fwww.recipecommunity.com.au%2Fesi%2Frecipes%2Fqlhgka4k-74388-924687-cfcd2-vtu0swek%3Flogin%3D0%26isAdminModerator%3D0&amp;media=https%3A%2F%2Fwww.recipecommunity.com.au%2Frecipes%2Fimage-cache%2F14f10ccb78abd1ff4990983bf0543a0b21473ed4%3Furl%3DaHR0cHM6Ly90bXJjdjMtYXUuczMuYW1hem9uYXdzLmNvbS9yZWNpcGVpbWFnZS9xbGhna2E0ay03NDM4OC05MjQ2ODctY2ZjZDItdnR1MHN3ZWsvMTlmMzkyYzctZWI2Ni00ZmE4LTk4MjUtMzRjYzc4Y2MyNzJmL21lZGl1bS9iYWJ5LXNhdXNhZ2Utc2Nyb2xscy5qcGc%253D%26adapter%3Dpinterest&amp;description=">
<img src="/

In [7]:
step = step_list[0]
img_string = step.img
operation = '#'+img_string['title']+'#'
step_icon = str(step).replace(str(img_string),operation)
step_icon_soup = BeautifulSoup(step_icon, 'html.parser')
step_icon_soup.text.strip()

'1. Place chicken, vegetables and coconut milk into TM bowl.\xa02. Cook for 10MIN - 100DEG -\xa0#Counter-clockwise operation#3. Pulverise for about 10SECS - SP7, checking there are no lumps. (or try for just 5SECS for a chunkier meal).\xa0The fats in coconut milk have been found to be close those in breast milk and thus is an ideal way to keep meat moist for your baby as well as being nutritious. \xa0This meal can be frozen (after using fresh chicken) and made thinner by adding more coconut milk, milk, breast milk if required.'

In [18]:
addtional_info_container = recipe_html_soup.find('div',class_= 'additional-info')
addtion_info_list = addtional_info_container.find_all('li')
#preparation time dowload
time_container = addtion_info_list[0]
preparation_time_container = time_container.find_all('div',class_='smallText')
print(preparation_time_container)
if (preparation_time_container):
    print('not none')
    self.preparation_time = preparation_time_container[0].find("span", id="preparation-time-final").text.strip()
    #cooking method
    cooking_method = preparation_time_container[1].text.strip()

[]


In [16]:
preparation_time_container == None

False

In [10]:
recipe_response = get('https://www.recipecommunity.com.au/baby-food-recipes/apple-kiwi-and-clove-puree-for-reusable-pouches/kl3uxs3v-0745c-386041-cfcd2-c0f9cw9z')
# recipe_response = get('https://www.recipecommunity.com.au/baby-food-recipes/baby-biscuits-sugar-free/vdttw1rs-09448-352970-cfcd2-icxafnee')
recipe_html_soup = BeautifulSoup(recipe_response.text, 'html.parser')

#download variations 
variations_container = recipe_html_soup.find('div',class_= 'col-sm-12 sidebar-box variant-box')
if (variations_container != None):
    variation_link = variations_container.a['href']
    variation_name = variations_container.a.text

#find recipe steps list
step_container = recipe_html_soup.find('ol', class_ = 'steps-list')
step_list = step_container.find_all('li')

#find ingredients list
ingredients_container = recipe_html_soup.find('div', class_ = 'ingredients')
ingredient_list = ingredients_container.find_all('li')

#process recipe step and ingredient list
# download_recipe_steps_ingredients(step_list,ingredient_list)

#accessories list download
accessories_container = recipe_html_soup.find('div', class_ = 'accessories-list')
if accessories_container == None:
    accessories = []
else:
    tools_list = accessories_container.find_all("meta")
    accessories = []
    for tool in tools_list:
        accessories.append(tool.get("content"))
    recipe_accessories = [accessories]
    
addtional_info_container = recipe_html_soup.find('div',class_= 'additional-info')
addtion_info_list = addtional_info_container.find_all('li')

In [11]:
time_container = addtion_info_list[0]
time = time_container.h5.text
preparation_time_container = time_container.find_all('div',class_='smallText')
preparation_time = preparation_time_container[0].find("span", id="preparation-time-final").text.strip()

NameError: name 'addtion_info_list' is not defined

In [19]:
iftested_container = recipe_html_soup.find('span','recipe-testing-status-text')
iftested = re.findall('\w+\stested', iftested_container.text)[0]
print(iftested)

not tested


In [137]:
cooking_method_container = preparation_time_container[1].text.strip()
print(cooking_method_container)

Baking/Cooking


In [140]:
machine_contianer = addtion_info_list[2]
print(machine_contianer)

<li>
<div class="step-container">
<div class="edit-icon"></div>
<span class="step-number">8</span>
<div class="appliances-list">
<ul>
<li class="media appliance-version no-border">
<div class="media-left">
<img alt="Appliance TM 31 image" class="media-object" data-holder-rendered="true" data-src="holder.js/50x50" src="//tmrcv3-common.s3.amazonaws.com/appliances/tm31_small.png" style=""/>
</div>
<div class="media-body">
<span class="smallText">
                Recipe is created for
              </span>
<h5 class="media-heading">TM 31</h5>
</div>
</li>
</ul>
</div>
</div>
</li>


In [142]:
machine_name = machine_contianer.h5.text
machine_addtional_info = machine_contianer.find('div',class_="margin-top-10")
if(machine_addtional_info!=None):
    machine_addtional_info = machine_addtional_info.text
print(machine_addtional_info)

None


In [159]:
addtional_categories_container = addtion_info_list[4]
addtional_categories_list =addtional_categories_container.find_all('a',class_='catText preventDefault')
addtional_categories_dic ={}
for category in addtional_categories_list:
    addtional_categories_dic[category.text] = category['href']
addtional_categories_dic

{}

In [160]:
tips_container = recipe_html_soup.find('div',class_='tips')
tips = tips_container.find('p').text
print(tips_container)


<div class="tips">
<div class="step-container">
<div class="edit-icon"></div>
<span class="step-number">11</span>
<h2 class="h3">Tip</h2>
<p><p> </p> </p>
</div>
</div>


In [161]:
tips = tips_container.find('p').text
print(tips)

In [155]:
portion= ingredients_container.find('p',class_='padding-bottom-10').text
print(portion)


 6 portion(s)

